In [7]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
# import seaborn as sns
from sklearn.preprocessing import OrdinalEncoder, LabelEncoder
from sklearn.model_selection import train_test_split

from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import SelectKBest,chi2,RFECV


from sklearn.feature_selection import SelectKBest,chi2,RFECV

from sklearn.preprocessing import OrdinalEncoder, LabelEncoder, KBinsDiscretizer
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer

import joblib

from sklearn.ensemble import RandomForestClassifier, StackingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix, mean_squared_error
%matplotlib inline

In [8]:
# Ignore warnings to see clean output
import warnings
warnings.filterwarnings("ignore")

In [9]:
# Load the data
df_FP=pd.read_csv("../raw_data/train_data_Total_FP.csv")
df_C=pd.read_csv("../raw_data/train_data_Captain.csv")
df_C_TD = pd.read_csv("../raw_data/test_data_Captain (1).csv")

In [10]:
# Check the first few rows of the data
df_FP.describe()

,match_id,season,batting_innings,bowling_innings,Starting_11,prev_runs,prev_balls,prev_sixes,prev_fours,prev_wickets,prev_conceded,prev_catches,prev_Dream Team,prev_Total_FP,prev_overs,prev_fielding_heroics,prev_duck,luck,Total_FP
count,21616.000000,21616.000000,21616.000000,21616.000000,21616.0,14699.000000,14699.000000,14699.000000,14699.000000,11335.000000,11335.000000,6682.000000,20939.000000,20939.000000,11335.000000,6682.000000,21616.000000,21616.000000,21616.000000
mean,11196.635918,2015.633975,1.497641,1.502359,4.0,19.622559,15.126267,0.759848,1.776720,0.916542,25.932510,1.022897,0.510292,39.281389,3.236568,0.275367,0.071521,49.633142,37.617274
std,6452.702186,4.663043,0.500006,0.500006,0.0,21.306417,13.636834,1.343462,2.307336,0.998369,10.675894,0.628788,0.499906,33.193384,1.017250,0.519249,0.257699,28.865971,31.775817
min,1.000000,2008.000000,1.000000,1.000000,4.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-4.000000,0.000000,0.000000,0.000000,0.000000,-4.000000
25%,5611.750000,2012.000000,1.000000,1.000000,4.0,3.000000,5.000000,0.000000,0.000000,0.000000,18.500000,1.000000,0.000000,12.000000,3.000000,0.000000,0.000000,25.000000,12.000000
50%,11195.500000,2015.000000,1.000000,2.000000,4.0,12.000000,11.000000,0.000000,1.000000,1.000000,26.000000,1.000000,1.000000,31.000000,4.000000,0.000000,0.000000,50.000000,30.000000
75%,16783.250000,2020.000000,2.000000,2.000000,4.0,29.000000,22.000000,1.000000,3.000000,1.000000,33.000000,1.000000,1.000000,57.000000,4.000000,0.000000,0.000000,75.000000,55.000000
max,22362.000000,2023.000000,2.000000,2.000000,4.0,175.000000,73.000000,17.000000,19.000000,6.000000,70.000000,5.000000,1.000000,306.000000,4.000000,4.000000,1.000000,99.000000,306.000000


In [11]:
# Check all the columns of the dataframe
df_FP.columns

Index(['match_id', 'season', 'match_name', 'home_team', 'away_team', 'venue',
       'batting_innings', 'bowling_innings', 'Player_name', 'Starting_11',
       'bowling_team_bowl', 'batting_team_bat', 'prev_runs', 'prev_balls',
       'prev_sixes', 'prev_fours', 'prev_wickets', 'prev_conceded',
       'prev_Overs_Bowled', 'prev_catches', 'prev_Dream Team', 'prev_Total_FP',
       'prev_overs', 'prev_fielding_heroics', 'prev_duck', 'luck', 'Total_FP'],
      dtype='object')

In [12]:
# Dropping nan columns
df_FP = df_FP[df_FP["prev_Total_FP"].notna()]
df_C = df_C[df_C["prev_Total_FP"].notna()]
print(df_C["match_id"].count())
print(df_FP["match_id"].count())

20939
20939


In [13]:
#Removing unnecessary columns

df_FP = df_FP.drop(
    ["Starting_11", "match_name", "bowling_innings", "prev_Overs_Bowled"], axis=1
)

df_C = df_C.drop(["Starting_11"], axis=1)
df_C_TD = df_C_TD.drop(["Starting_11"], axis=1)
df_C


,match_id,season,match_name,home_team,away_team,venue,batting_innings,bowling_innings,Player_name,bowling_team_bowl,...,prev_conceded,prev_Overs_Bowled,prev_catches,prev_Dream Team,prev_Total_FP,prev_overs,prev_fielding_heroics,prev_duck,luck,Captain/Vice Captain
1,2,2008,MI v DC,MI,DC,"Dr DY Patil Sports Academy, Mumbai",2.0,1.0,AB de Villiers,NaN,...,NaN,NaN,NaN,1.0,32.0,NaN,NaN,0,35,NC
2,3,2008,DC v CSK,DC,CSK,"Arun Jaitley Stadium, Delhi",1.0,2.0,AB de Villiers,NaN,...,NaN,NaN,1.0,0.0,37.0,NaN,0.0,0,79,NC
3,4,2008,RR v DC,RR,DC,"Sawai Mansingh Stadium, Jaipur",1.0,2.0,AB de Villiers,NaN,...,NaN,NaN,2.0,0.0,18.0,NaN,0.0,1,33,NC
4,5,2008,KKR v DC,KKR,DC,"Eden Gardens, Kolkata",2.0,1.0,AB de Villiers,NaN,...,NaN,NaN,0.0,1.0,36.0,NaN,1.0,0,82,NC
5,6,2008,RCB v DC,RCB,DC,"M.Chinnaswamy Stadium, Bengaluru",2.0,1.0,AB de Villiers,NaN,...,NaN,NaN,0.0,0.0,13.0,NaN,1.0,0,54,NC
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21610,22357,2023,PBKS v RR,PBKS,RR,"Himachal Pradesh Cricket Association Stadium, ...",1.0,2.0,substitute_fielder,NaN,...,NaN,NaN,NaN,0.0,12.0,NaN,NaN,0,10,NC
21611,22358,2023,PBKS v RR,PBKS,RR,"Himachal Pradesh Cricket Association Stadium, ...",2.0,1.0,substitute_fielder,NaN,...,NaN,NaN,NaN,0.0,20.0,NaN,NaN,0,3,NC
21612,22359,2023,MI v SRH,MI,SRH,"Wankhede Stadium, Mumbai",2.0,1.0,substitute_fielder,NaN,...,NaN,NaN,NaN,0.0,12.0,NaN,NaN,0,80,NC
21613,22360,2023,RCB v GT,RCB,GT,"M.Chinnaswamy Stadium, Bengaluru",1.0,2.0,substitute_fielder,NaN,...,NaN,NaN,NaN,0.0,12.0,NaN,NaN,0,4,NC


In [14]:
df_C["Captain/Vice Captain"].value_counts()


Captain/Vice Captain
NC    19333
VC      815
C       791
Name: count, dtype: int64

In [15]:
# Initialize LabelEncoder
label_encoder = LabelEncoder()

# Fit and transform the list of teams
df_FP["home_team"] = label_encoder.fit_transform(df_FP["home_team"])
df_FP["batting_team_bat"] = label_encoder.fit_transform(df_FP["batting_team_bat"])
df_FP["bowling_team_bowl"] = label_encoder.fit_transform(df_FP["bowling_team_bowl"])
df_FP["away_team"] = label_encoder.fit_transform(df_FP["away_team"])
df_FP["venue"] = label_encoder.fit_transform(df_FP["venue"])
df_FP["Player_name"] = label_encoder.fit_transform(df_FP["Player_name"])
df_C["Captain/Vice Captain"] = label_encoder.fit_transform(df_C["Captain/Vice Captain"])
df_C["Captain/Vice Captain"].value_counts()

Captain/Vice Captain
1    19333
2      815
0      791
Name: count, dtype: int64

In [16]:
# 1 == NC, 2 == VC, 3 == C

In [17]:
# Fill nan values
df_FP["prev_overs"].fillna(-0.1, inplace=True)
df_FP["balls"] = (
    6 * df_FP["prev_overs"].astype(int)
    + (df_FP["prev_overs"] - df_FP["prev_overs"].astype(int)) * 10
)
df_FP["balls"].value_counts()
df_FP["prev_conceded"].fillna(-10, inplace=True)
df_FP["economy"] = (df_FP["prev_conceded"] + 1e-5) / (1e-5 + df_FP["balls"]) * 6
df_FP["economy"].value_counts()
df_FP["prev_runs"].fillna(1.0, inplace=True)
df_FP["prev_balls"].fillna(-2.0, inplace=True)

df_FP["strikerate"] = (1 + df_FP["prev_runs"]) / (df_FP["prev_balls"] + 1) * 100
print(df_FP["strikerate"].isnull().sum())
df_FP["strikerate"].value_counts()
df_FP.info()
df_FP["prev_fours"].fillna(-1, inplace=True)
df_FP["prev_sixes"].fillna(-1, inplace=True)
df_FP["boundaries"] = df_FP["prev_fours"] + df_FP["prev_sixes"]
df_FP["boundaries"].value_counts()
df_FP["prev_wickets"].fillna(-1, inplace=True)
df_FP["prev_fielding_heroics"].fillna(0, inplace=True)
df_FP["prev_catches"].fillna(0, inplace=True)
df_FP["Total_FP"].value_counts().sort_index()
df_FP = df_FP.drop(
    ["prev_overs", "prev_conceded", "prev_fours", "prev_sixes", "prev_runs"], axis=1
)

# Number of nan values in each column
print((df_FP.isna().sum()).count())

# Splitting data into train and test
x_train, x_test, y_train, y_test = train_test_split(
    df_FP.drop("Total_FP", axis=1),
    pd.concat([df_C["Captain/Vice Captain"], df_FP["Total_FP"]], axis=1),
    test_size=0.35,
    random_state=42,
)

0
<class 'pandas.core.frame.DataFrame'>
Index: 20939 entries, 1 to 21614
Data columns (total 26 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   match_id               20939 non-null  int64  
 1   season                 20939 non-null  int64  
 2   home_team              20939 non-null  int32  
 3   away_team              20939 non-null  int32  
 4   venue                  20939 non-null  int32  
 5   batting_innings        20939 non-null  float64
 6   Player_name            20939 non-null  int32  
 7   bowling_team_bowl      20939 non-null  int32  
 8   batting_team_bat       20939 non-null  int32  
 9   prev_runs              20939 non-null  float64
 10  prev_balls             20939 non-null  float64
 11  prev_sixes             14699 non-null  float64
 12  prev_fours             14699 non-null  float64
 13  prev_wickets           11335 non-null  float64
 14  prev_conceded          20939 non-null  float64
 15  prev_

In [18]:
#Checking the split
y_test["Captain/Vice Captain"].value_counts()

Captain/Vice Captain
1    6751
0     294
2     284
Name: count, dtype: int64

In [19]:
x_train.columns

Index(['match_id', 'season', 'home_team', 'away_team', 'venue',
       'batting_innings', 'Player_name', 'bowling_team_bowl',
       'batting_team_bat', 'prev_balls', 'prev_wickets', 'prev_catches',
       'prev_Dream Team', 'prev_Total_FP', 'prev_fielding_heroics',
       'prev_duck', 'luck', 'balls', 'economy', 'strikerate', 'boundaries'],
      dtype='object')

In [20]:
print(x_train['season'].unique())
print(x_test['season'].unique())

[2008 2019 2017 2015 2010 2011 2016 2013 2023 2009 2012 2021 2018 2020
 2014 2022]
[2016 2011 2020 2015 2023 2014 2017 2022 2013 2010 2009 2012 2019 2018
 2008 2021]


In [21]:
x_train.isnull().sum()

match_id                 0
season                   0
home_team                0
away_team                0
venue                    0
batting_innings          0
Player_name              0
bowling_team_bowl        0
batting_team_bat         0
prev_balls               0
prev_wickets             0
prev_catches             0
prev_Dream Team          0
prev_Total_FP            0
prev_fielding_heroics    0
prev_duck                0
luck                     0
balls                    0
economy                  0
strikerate               0
boundaries               0
dtype: int64

In [22]:
print(len(y_train["Captain/Vice Captain"]))
print(len(x_train))
print(y_train.shape)
print(x_train.shape)

13610
13610
(13610, 2)
(13610, 21)


In [23]:
y_train.reset_index(drop=True, inplace=True)
y_train

,Captain/Vice Captain,Total_FP
0,1,25.0
1,2,94.0
2,1,12.0
3,1,33.0
4,1,41.0
...,...,...
13605,1,45.0
13606,1,11.0
13607,1,13.0
13608,1,23.0


In [24]:
x_train.reset_index(drop=True, inplace=True)
x_train

,match_id,season,home_team,away_team,venue,batting_innings,Player_name,bowling_team_bowl,batting_team_bat,prev_balls,...,prev_catches,prev_Dream Team,prev_Total_FP,prev_fielding_heroics,prev_duck,luck,balls,economy,strikerate,boundaries
0,16795,2008,7,12,7,2.0,493,7,7,16.0,...,0.0,1.0,56.0,0.0,0,29,6.0,15.999983,205.882353,7.0
1,11538,2019,0,7,18,2.0,346,0,0,-2.0,...,0.0,1.0,29.0,0.0,0,30,24.0,7.499999,-200.000000,-2.0
2,11209,2019,1,8,0,1.0,334,14,8,12.0,...,2.0,1.0,57.0,0.0,0,36,-1.0,60.000540,207.692308,3.0
3,19032,2017,8,11,14,1.0,556,14,11,54.0,...,0.0,1.0,113.0,0.0,0,73,-1.0,60.000540,154.545455,10.0
4,1973,2015,0,8,18,2.0,63,8,8,-2.0,...,0.0,0.0,27.0,0.0,0,20,24.0,10.249998,-200.000000,-2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13605,12088,2016,13,2,25,2.0,359,13,13,14.0,...,1.0,1.0,83.0,0.0,0,79,24.0,9.999998,213.333333,4.0
13606,12810,2020,10,1,10,2.0,379,10,10,-2.0,...,0.0,1.0,41.0,0.0,0,88,24.0,9.249999,-200.000000,-2.0
13607,5799,2009,1,12,20,1.0,185,14,1,11.0,...,0.0,0.0,12.0,0.0,0,37,-1.0,60.000540,75.000000,0.0
13608,928,2011,0,1,18,2.0,27,1,14,6.0,...,0.0,0.0,12.0,0.0,0,15,24.0,7.749999,114.285714,1.0


In [25]:
data_ag = pd.concat([x_train, y_train['Captain/Vice Captain']], axis=1)

In [26]:
data_ag

,match_id,season,home_team,away_team,venue,batting_innings,Player_name,bowling_team_bowl,batting_team_bat,prev_balls,...,prev_Dream Team,prev_Total_FP,prev_fielding_heroics,prev_duck,luck,balls,economy,strikerate,boundaries,Captain/Vice Captain
0,16795,2008,7,12,7,2.0,493,7,7,16.0,...,1.0,56.0,0.0,0,29,6.0,15.999983,205.882353,7.0,1
1,11538,2019,0,7,18,2.0,346,0,0,-2.0,...,1.0,29.0,0.0,0,30,24.0,7.499999,-200.000000,-2.0,2
2,11209,2019,1,8,0,1.0,334,14,8,12.0,...,1.0,57.0,0.0,0,36,-1.0,60.000540,207.692308,3.0,1
3,19032,2017,8,11,14,1.0,556,14,11,54.0,...,1.0,113.0,0.0,0,73,-1.0,60.000540,154.545455,10.0,1
4,1973,2015,0,8,18,2.0,63,8,8,-2.0,...,0.0,27.0,0.0,0,20,24.0,10.249998,-200.000000,-2.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13605,12088,2016,13,2,25,2.0,359,13,13,14.0,...,1.0,83.0,0.0,0,79,24.0,9.999998,213.333333,4.0,1
13606,12810,2020,10,1,10,2.0,379,10,10,-2.0,...,1.0,41.0,0.0,0,88,24.0,9.249999,-200.000000,-2.0,1
13607,5799,2009,1,12,20,1.0,185,14,1,11.0,...,0.0,12.0,0.0,0,37,-1.0,60.000540,75.000000,0.0,1
13608,928,2011,0,1,18,2.0,27,1,14,6.0,...,0.0,12.0,0.0,0,15,24.0,7.749999,114.285714,1.0,1


In [1]:
from autogluon.tabular import TabularPredictor

c:\Users\spec5\.conda\envs\ag\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [22]:

# Fit the model using AutoGluon
predictor = TabularPredictor(label="Captain/Vice Captain").fit(
    data_ag.drop(["balls", "economy", "strikerate", "boundaries"], axis=1),
    presets="high_quality",
    ag_args_fit={"num_gpus": 1},
    ag_args_ensemble=dict(fold_fitting_strategy="sequential_local")
)

No path specified. Models will be saved in: "AutogluonModels\ag-20240602_175553"
Presets specified: ['high_quality']
Stack configuration (auto_stack=True): num_stack_levels=1, num_bag_folds=8, num_bag_sets=1
Dynamic stacking is enabled (dynamic_stacking=True). AutoGluon will try to determine whether the input data is affected by stacked overfitting and enable or disable stacking as a consequence.
Detecting stacked overfitting by sub-fitting AutoGluon on the input data. That is, copies of AutoGluon will be sub-fit on subset(s) of the data. Then, the holdout validation data is used to detect stacked overfitting.
Sub-fit(s) time limit is: 3600 seconds.
Starting holdout-based sub-fit for dynamic stacking. Context path is: AutogluonModels\ag-20240602_175553/ds_sub_fit/sub_fit_ho.
2024-06-02 23:25:54,117	INFO util.py:159 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
Beginning AutoGluon training ... Time limit 

In [27]:
x_test

,match_id,season,home_team,away_team,venue,batting_innings,Player_name,bowling_team_bowl,batting_team_bat,prev_balls,...,prev_catches,prev_Dream Team,prev_Total_FP,prev_fielding_heroics,prev_duck,luck,balls,economy,strikerate,boundaries
5740,5961,2016,4,11,11,1.0,186,14,11,40.0,...,0.0,1.0,47.0,0.0,0,26,-1.0,60.000540,85.365854,4.0
20252,20954,2011,10,4,17,2.0,617,10,10,34.0,...,0.0,1.0,56.0,0.0,0,95,18.0,5.666667,114.285714,4.0
11224,11631,2020,10,0,10,1.0,349,10,10,13.0,...,1.0,1.0,47.0,0.0,0,13,12.0,10.499996,85.714286,1.0
4779,4959,2015,10,8,17,1.0,161,14,10,20.0,...,0.0,1.0,43.0,1.0,0,99,-1.0,60.000540,114.285714,4.0
11599,12020,2023,3,7,21,1.0,356,3,14,-2.0,...,0.0,1.0,54.0,0.0,0,79,24.0,7.749999,-200.000000,-2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2266,2356,2023,3,6,21,2.0,78,6,14,-2.0,...,0.0,0.0,-2.0,0.0,0,92,12.0,13.999993,-200.000000,-2.0
4008,4160,2016,8,10,24,2.0,143,14,8,6.0,...,1.0,0.0,24.0,0.0,0,82,-1.0,60.000540,171.428571,1.0
4624,4799,2019,12,8,28,2.0,155,12,12,-2.0,...,0.0,1.0,27.0,0.0,0,21,24.0,10.749998,-200.000000,-2.0
6833,7104,2009,8,4,16,2.0,217,4,14,9.0,...,0.0,0.0,21.0,0.0,0,95,18.0,4.333334,100.000000,1.0


In [30]:
# Refit complete, total runtime = 156.59s ... Best model: "WeightedEnsemble_L2_FULL"
# TabularPredictor saved. To load, use: predictor = TabularPredictor.load("AutogluonModels\ag-20240602_175553")
predictor = TabularPredictor.load("AutogluonModels/ag-20240602_175553/")
# Make predictions on the test data
y_pred = predictor.predict(x_test)

# Calculate accuracy
accuracy = (y_pred == y_test["Captain/Vice Captain"]).mean()
print(f"Accuracy: {accuracy}")

Accuracy: 0.9211352162641561


In [31]:
df_C_TD

,match_id,season,match_name,home_team,away_team,venue,batting_innings,bowling_innings,Player_name,bowling_team_bowl,...,prev_wickets,prev_conceded,prev_Overs_Bowled,prev_catches,prev_Dream Team,prev_Total_FP,prev_overs,prev_fielding_heroics,prev_duck,luck
0,8486,2009,RCB v RR,RCB,RR,"Newlands, Cape Town",2.0,1.0,Kamran Khan,RR,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,96
1,21272,2022,RCB v GT,RCB,GT,"Wankhede Stadium, Mumbai",2.0,1.0,Wanindu Hasaranga de Silva,RCB,...,2.0,15.0,"[7, 9, 11, 17]",1.0,1.0,69.0,4.0,0.0,0,40
2,16353,2023,LSG v DC,LSG,DC,"Bharat Ratna Shri Atal Bihari Vajpayee Ekana Cricket Stadium, Lucknow",2.0,1.0,Rovman Powell,NaN,...,NaN,NaN,NaN,NaN,1.0,60.0,NaN,NaN,0,51
3,13436,2017,RCB v DC,RCB,DC,"M.Chinnaswamy Stadium, Bengaluru",1.0,2.0,Pawan Negi,RCB,...,1.0,19.0,"[7, 9, 15]",NaN,1.0,33.0,3.0,NaN,0,41
4,1323,2023,RCB v CSK,RCB,CSK,"M.Chinnaswamy Stadium, Bengaluru",1.0,2.0,Ambati Rayudu,NaN,...,NaN,NaN,NaN,NaN,0.0,5.0,NaN,NaN,0,88
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
704,11809,2022,GT v LSG,GT,LSG,"Wankhede Stadium, Mumbai",2.0,1.0,Mohammed Shami,GT,...,3.0,21.0,"[1, 3, 17, 19]",NaN,1.0,95.0,4.0,NaN,0,78
705,15583,2022,KKR v RR,KKR,RR,"Wankhede Stadium, Mumbai",2.0,1.0,Rinku Singh,NaN,...,NaN,NaN,NaN,NaN,1.0,32.0,NaN,NaN,0,39
706,14560,2020,SRH v RR,SRH,RR,Dubai International Cricket Stadium,2.0,1.0,Rahul Tewatia,RR,...,1.0,6.0,[10],NaN,1.0,47.0,1.0,NaN,0,79
707,4947,2014,DC v PBKS,DC,PBKS,"Arun Jaitley Stadium, Delhi",1.0,2.0,Dinesh Karthik,NaN,...,NaN,NaN,NaN,0.0,0.0,19.0,NaN,1.0,0,95
